In [ ]:
%pip install sshtunnel
%pip install requests
%pip install psycopg2
%pip install pgvector
%pip install python-dotenv

In [ ]:
%load_ext dotenv
%dotenv

CREATE HELPER FOR THE DB CONNECTION

In [ ]:
from sshtunnel import SSHTunnelForwarder
import requests
import json
import os
import pgvector
import psycopg2
from pgvector.psycopg import register_vector
import json
def get_db_connection():
    # Setting up the SSH tunnel with tunnel credentials
    REMOTE_HOST = os.getenv("REMOTE_HOST")
    REMOTE_SSH_PORT = int(os.getenv("REMOTE_SSH_PORT"))
    PORT = int(os.getenv("PORT"))
    SSH_KEYFILE = os.getenv("SSH_KEYFILE")
    SSH_USERNAME =  os.getenv("SSH_USERNAME")

    server = SSHTunnelForwarder(
        ssh_address_or_host=(REMOTE_HOST, REMOTE_SSH_PORT),
        ssh_username= SSH_USERNAME,
        ssh_pkey=SSH_KEYFILE,
        # Key part! Connect to AWS_HOST through the tunnel.
        remote_bind_address=('localhost', PORT)
    )
    server.start()
    print("server connected")

    conn_str = f"dbname=postgres host=localhost port={server.local_bind_port} user=postgres password={os.getenv('DB_PASSWORD')}"
    return psycopg2.connect(conn_str)

CREATE VERTICES FROM CASES TABLE

In [ ]:
import psycopg2

# Database connection parameters
conn_params = {
    'dbname': 'mluk-genai-demo',
    'user': 'azureuser',
    'password': 'Caroot-gen-0',
    'host': '4.248.28.54',
    'port': '5432'
}

# Connect to the PostgreSQL database
conn = psycopg2.connect(**conn_params)
cursor = conn.cursor()

# Create a graph if it doesn't exist
cursor.execute("SELECT create_graph('playground_case_graph');")

# Fetch data from the cases_playground table
cursor.execute("SELECT id, data->>'id' AS case_id, data->>'name' AS name FROM cases_playground;")
records = cursor.fetchall()

# Create vertices using Cypher
for record in records:
    id, case_id, name = record
    cypher_query = f"""
    SELECT * FROM cypher('playground_case_graph', $$
        CREATE (c:Case {{id: {id}, case_id: '{case_id}', name: '{name}'}})
    $$) AS (v agtype);
    """
    cursor.execute(cypher_query)

# Commit the transaction
conn.commit()

# Close the cursor and connection
cursor.close()
conn.close()

print("Vertices created successfully.")
